<a href="https://colab.research.google.com/github/SomeiLam/langchain-example/blob/main/LangChain_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U wikipedia
!pip install langchain
!pip install openai
!pip install langchain_community
!pip install langchain-experimental
!pip install -U langchain-openai

In [25]:
import os
import openai
from google.colab import userdata
api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key
openai.api_key = os.environ['OPENAI_API_KEY']
llm_model = "gpt-4o-mini"

In [10]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.utilities.python import PythonREPL
from langchain_openai import ChatOpenAI

In [26]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [11]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [14]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True,
)

# use it as before:
resp = agent.run("Compute the square root of 144.")

<ipython-input-14-7b44ccaf8097>:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resp = agent.run("Compute the square root of 144.")




> Entering new AgentExecutor chain...
Thought: I can use the Calculator tool to compute the square root of 144.

Action:
```
{
  "action": "Calculator",
  "action_input": "sqrt(144)"
}
```
Observation: Answer: 12.0
Thought:I now know the final answer

Final Answer: 12.0

> Finished chain.


In [ ]:
!pip install langgraph

In [27]:
from typing_extensions import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_core.output_parsers import StrOutputParser

In [28]:
python_tool = PythonREPLTool()

In [29]:
# ThinkResult schema
class ThinkResult(TypedDict):
    thought: str
    action: str

# Build a prompt that instructs JSON output
react_prompt = (
    ChatPromptTemplate.from_messages([
        {"role":"system","content":
         "You are an agent that thinks step-by-step and then chooses an action."},
        {"role":"user","content":
         "INPUT: {input}\n\n"
         "Respond with a JSON object matching this schema:\n"
         "```json\n"
         "{{\n"
         '  "thought": string,   // your internal reasoning\n'
         '  "action": string     // "python:<code>" to run code, or "final:<text>"\n'
         "}}\n"
         "```"
        }
    ])
    # Tell the LLM to output that JSON schema
    | llm.with_structured_output(ThinkResult)
)

In [30]:
# State schema
class State(TypedDict):
    input: str
    thought: str
    action: str
    observation: str
    final: Literal[True]

In [34]:
# Node functions
async def think(state: State, config: RunnableConfig) -> ThinkResult:
    # Returns a dict with exactly "thought" and "action"
    result: ThinkResult = await react_prompt.ainvoke(state["input"], config)
    return result

async def run_tool(state: State, config: RunnableConfig):
    action = state["action"]
    if action.startswith("python:"):
        code = action[len("python:"):]
        observation = python_tool.run(code)
    else:
        observation = "<no tool run>"
    return {"observation": observation}

async def finish(state: State, config: RunnableConfig):
    return {"final": True}

def select_next(state: State):
    # always go from think→run_tool→END
    if not state.get("observation"):
        return "run_tool"
    return "finish"

In [35]:
# Assemble the graph
graph = StateGraph(State)
graph.add_node("think", think)
graph.add_node("run_tool", run_tool)
graph.add_node("finish", finish)
graph.add_conditional_edges("think", select_next)
graph.add_edge(START, "think")
graph.add_edge("run_tool", END)
graph.add_edge("finish", END)

In [37]:
# Compile and run
agent = graph.compile()
result = await agent.ainvoke({"input": "Compute 12 * 12 using python."})
result

{'input': 'Compute 12 * 12 using python.',
 'thought': 'To compute 12 * 12 in Python, I will write a simple multiplication expression. The result of this operation should be 144.',
 'action': 'python:12 * 12',
 'observation': ''}

In [39]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [48]:
agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [49]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question)



> Entering new AgentExecutor chain...
Thought: I need to find out what book Tom M. Mitchell has written. I will look up his works on Wikipedia to gather this information.
Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```


/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past president of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Ensemble learning
Summary: In statistics and machine learning, ensemble methods use multiple learning algorithms to obtai

In [50]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [51]:
customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [52]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")



> Entering new AgentExecutor chain...
To sort the customers by last name and then by first name, I will use Python's built-in sorting capabilities. I will sort the list of customers using a custom key that sorts primarily by the last name and secondarily by the first name. 

Action: Python_REPL  
Action Input: `customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]; sorted_customers = sorted(customers, key=lambda x: (x[0], x[1])); print(sorted_customers)`  
Observation: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]

Thought:I now know the final answer
Final Answer: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]

> Finished chain.


"[['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]"

In [53]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an

In [ ]:
!pip install DateTime

In [55]:
from langchain.agents import tool
from datetime import date

In [56]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [57]:
agent= initialize_agent(
    tools + [time],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [58]:
try:
    result = agent("whats the date today?")
except:
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I need to find out today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2025-05-08
Thought:I now know the final answer
Final Answer: 2025-05-08

> Finished chain.
